# Ames Data Exercise for 7.9 Lecture

*By Rob Finch*

### The Problem (from Lesson 5, Notebook 3):

Return to your Ames Data.  We have covered a lot of ground today, so let's summarize the things we could do to improve the performance of our original model that compared the Above Ground Living Area to the Logarithm of the Sale Price.
<div class="alert alert-info" role="alert">
1. Clean data, drop missing values
2. Transform data, code variables using either ordinal values or OneHotEncoder methods
3. Create more features from existing features
4. Split our data into testing and training sets
5. Normalize quantitative features
6. Use Regularized Regression methods and Polynomial regression to improve performance of model
</div>
Can you use some or all of these ideas to improve upon your initial model?

## Section 1: Cleaning Data

In [34]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
ames = pd.read_csv('data/ames_housing.csv')
ames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

**Here, I'm filling each object type cell with "none."  I checked the data in each column and the data appears to be dependent on the property, so using "None" with each of these is a safe bet.**

In [35]:
ames['Alley'] = ames['Alley'].fillna('None')
ames['MasVnrType'] = ames['MasVnrType'].fillna('None')
ames['BsmtQual'] = ames['BsmtQual'].fillna('None')
ames['BsmtCond'] = ames['BsmtCond'].fillna('None')
ames['BsmtExposure'] = ames['BsmtExposure'].fillna('None')
ames['BsmtFinType1'] = ames['BsmtFinType1'].fillna('None')
ames['BsmtFinType2'] = ames['BsmtFinType2'].fillna('None')
ames['Electrical'] = ames['Electrical'].fillna('None')
ames['FireplaceQu'] = ames['FireplaceQu'].fillna('None')
ames['GarageType'] = ames['GarageType'].fillna('None')
ames['GarageFinish'] = ames['GarageFinish'].fillna('None')
ames['GarageQual'] = ames['GarageQual'].fillna('None')
ames['GarageCond'] = ames['GarageCond'].fillna('None')
ames['PoolQC'] = ames['PoolQC'].fillna('None')
ames['Fence'] = ames['Fence'].fillna('None')
ames['MiscFeature'] = ames['MiscFeature'].fillna('None')

**Similarly to the object type series with missing data, the three float series with missing values are because they don't have the data (lot frontage) or a feature doesn't apply (MasVnrArea & GarageYrBuilt).**

In [36]:
ames['LotFrontage'] = ames['LotFrontage'].fillna(0.0)
ames['MasVnrArea'] = ames['MasVnrArea'].fillna(0.0)
ames['GarageYrBlt'] = ames['GarageYrBlt'].fillna(0.0)

**Now we check the data again and see that all columns/rows are filled with a value so we can do our analysis.**

In [37]:
ames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non

## Section 2: Transforming the dataset

Here, I've gone ahead and ranked some of the columns in ordinal values where applicable.  Note that, rather than using the *.replace* method in the notebook exampes, I chose to add new columns for these and preserved the original data as-is, via the *.map* method.

For other data series, there was either not enough information to rank the values appropriately, or the series did not belong in a set order.

In [38]:
ames['StreetOrd'] = ames['Street'].map({'Grvl': 1, 'Pave': 2})
ames['UtilitiesOrd'] = ames['Utilities'].map({'NoSeWa': 1, 'AllPub': 2})
ames['ExterQualOrd'] = ames['ExterQual'].map({"Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5})
ames['ExterCondOrd'] = ames['ExterCond'].map({"Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5})
ames['BsmtQualOrd'] = ames['BsmtQual'].map({"Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5})
ames['BsmtCondOrd'] = ames['BsmtCond'].map({"Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5})
ames['KitchenQualOrd'] = ames['KitchenQual'].map({"Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5})
ames['GarageQualOrd'] = ames['GarageQual'].map({'NA': 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5})
ames['GarageCondOrd'] = ames['GarageCond'].map({'NA': 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5})

## Section 3: Adding New Features

Below, I've added composite columns that take both the condition and quality values of each property into account when evaluating.

In [39]:
ames['BasementOverall'] = ames['BsmtCondOrd'] * ames['BsmtQualOrd']
ames['Overall'] = ames['OverallQual'] * ames['OverallCond']
ames['GarageOverall'] = ames['GarageCondOrd'] * ames['GarageQualOrd']
ames['ExternalOverall'] = ames['ExterCondOrd'] * ames['ExterQualOrd']

### Now let's check the initial correlations.  As my variable to model against, I'm working with home sale price.

In [40]:
corr = ames.corr()
corr.SalePrice

Id                -0.021917
MSSubClass        -0.084284
LotFrontage        0.209624
LotArea            0.263843
OverallQual        0.790982
OverallCond       -0.077856
YearBuilt          0.522897
YearRemodAdd       0.507101
MasVnrArea         0.472614
BsmtFinSF1         0.386420
BsmtFinSF2        -0.011378
BsmtUnfSF          0.214479
TotalBsmtSF        0.613581
1stFlrSF           0.605852
2ndFlrSF           0.319334
LowQualFinSF      -0.025606
GrLivArea          0.708624
BsmtFullBath       0.227122
BsmtHalfBath      -0.016844
FullBath           0.560664
HalfBath           0.284108
BedroomAbvGr       0.168213
KitchenAbvGr      -0.135907
TotRmsAbvGrd       0.533723
Fireplaces         0.466929
GarageYrBlt        0.261366
GarageCars         0.640409
GarageArea         0.623431
WoodDeckSF         0.324413
OpenPorchSF        0.315856
EnclosedPorch     -0.128578
3SsnPorch          0.044584
ScreenPorch        0.111447
PoolArea           0.092404
MiscVal           -0.021190
MoSold             0

** After running the initial correlation against home sale price, the * OverallQual * variable appears to correlate most strongly and should be a part of my initial model: **


In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X = ames[['OverallQual']]
y = ames[['SalePrice']]

lr = LinearRegression()
lr.fit(X, y)
pred = lr.predict(X)
mse = mean_squared_error(pred, y)
rmse = np.sqrt(mse)
print('MSE: {:.4f}'.format(mse), '\nRMSE: {:.4f}'.format(rmse))

MSE: 2360934371.5670 
RMSE: 48589.4471


With a RMSE of 48,589, let's see what we can do improve our model.

# Method 2: Polynomial Features

Here we are adding a 2nd degree to the equation using PolynomialFeatures.

In [42]:
pipe = make_pipeline(PolynomialFeatures(2),
                     LinearRegression())

X = ames[['OverallQual']]
y = ames[['SalePrice']]

pipe.fit(X, y)

Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [43]:
predictions_square = pipe.predict(X)

In [44]:
np.sqrt(mean_squared_error(predictions_square, y))

45032.69069407139

With adding a polynomial feature of 2, we improved the RMSE a little bit, but not a lot.  Let's explore what cubing it does:

In [45]:
pipe = make_pipeline(PolynomialFeatures(3),
                     LinearRegression())

X = ames[['OverallQual']]
y = ames[['SalePrice']]

pipe.fit(X, y)

Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=3, include_bias=True, interaction_only=False)), ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [46]:
predictions_cube = pipe.predict(X)

In [47]:
np.sqrt(mean_squared_error(predictions_cube, y))

44818.65310265686

The improvement here is more negligible still.  I am adding 'GrLivArea' and 'GarageCars' to the X side of the graph to enhance the model as well.

In [48]:
pipe = make_pipeline(PolynomialFeatures(3),
                     LinearRegression())

X_new = ames[['OverallQual', 'GrLivArea', 'GarageCars']]
y = ames[['SalePrice']]

pipe.fit(X_new, y)

Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=3, include_bias=True, interaction_only=False)), ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [49]:
predictions_cube_multi = pipe.predict(X_new)

In [50]:
np.sqrt(mean_squared_error(predictions_cube_multi, y))

33716.0258764879

We are making *some* progress here, but the RMSE is still higher than we would like it to be.

Now, we are going to use the GridSearch function to enhance our model and find which polynomial features are the best.

In [61]:
from sklearn.model_selection import GridSearchCV

pipe = make_pipeline(PolynomialFeatures(), LinearRegression())
degrees = {'polynomialfeatures__degree': [i for i in range(1, 10)]}
grid = GridSearchCV(pipe, param_grid=degrees, cv=5)
grid.fit(X.values.reshape(-1,1), y)
best = grid.best_estimator_

print(best)

Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=5, include_bias=True, interaction_only=False)), ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])


As we can see from the above, the best degree for the polynomial features is 5.  Plugging that into the model once again, we get:

In [62]:
pipe = make_pipeline(PolynomialFeatures(5),
                     LinearRegression())

X_new = ames[['OverallQual', 'GrLivArea', 'GarageCars']]
y = ames[['SalePrice']]

pipe.fit(X_new, y)

Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=5, include_bias=True, interaction_only=False)), ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [63]:
predictions_cube_5 = pipe.predict(X_new)
np.sqrt(mean_squared_error(predictions_cube_5, y))

33040.14424608441

We get a marginally improved RMSE.

# Method 3: Train-Test Split

In [64]:
from sklearn.model_selection import train_test_split

X_new_train, X_new_test, y_train, y_test = train_test_split(X_new, y)

In [65]:
lm = LinearRegression()
lm.fit(X_new_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [66]:
predictions_new = lm.predict(X_new_test)
mse = mean_squared_error(y_test, predictions_new)
rmse = np.sqrt(mse)
print("MSE: ", mse, "\nRMSE: ", rmse)

MSE:  1855815818.6992543 
RMSE:  43079.18080348388


Here we will combine the split with some polynomial functionality (again with a degree of 5 from our grid example) to see if we can improve:

In [75]:
pipe = make_pipeline(PolynomialFeatures(5),
                     LinearRegression())

X_new = ames[['OverallQual', 'GrLivArea', 'GarageCars']]
y = ames[['SalePrice']]

pipe.fit(X_new_train, y_train)

Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=5, include_bias=True, interaction_only=False)), ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [76]:
predictions_cube_multi_test = pipe.predict(X_new_test)
mse = mean_squared_error(y_test, predictions_cube_multi_test)
rmse = np.sqrt(mse)
print("MSE: ", mse, "\nRMSE: ", rmse)

MSE:  1273843436.6018903 
RMSE:  35690.94334144014


## Trying StandardScaler

In [77]:
ames_test = ames.select_dtypes('int64')

In [78]:
ames_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 42 columns):
Id                 1460 non-null int64
MSSubClass         1460 non-null int64
LotArea            1460 non-null int64
OverallQual        1460 non-null int64
OverallCond        1460 non-null int64
YearBuilt          1460 non-null int64
YearRemodAdd       1460 non-null int64
BsmtFinSF1         1460 non-null int64
BsmtFinSF2         1460 non-null int64
BsmtUnfSF          1460 non-null int64
TotalBsmtSF        1460 non-null int64
1stFlrSF           1460 non-null int64
2ndFlrSF           1460 non-null int64
LowQualFinSF       1460 non-null int64
GrLivArea          1460 non-null int64
BsmtFullBath       1460 non-null int64
BsmtHalfBath       1460 non-null int64
FullBath           1460 non-null int64
HalfBath           1460 non-null int64
BedroomAbvGr       1460 non-null int64
KitchenAbvGr       1460 non-null int64
TotRmsAbvGrd       1460 non-null int64
Fireplaces         1460 non-null in

In [83]:
X = ames_test.drop('SalePrice', axis=1)
y = ames_test['SalePrice']

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [87]:
X_train_scaled = StandardScaler().fit_transform(X_train)
X_test_scaled = StandardScaler().fit_transform(X_test)

In [89]:
from sklearn.linear_model import Ridge
ridge = Ridge()
params = {'alpha': [0.1, 0.7, 1.0, 3.0, 10]}

In [92]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(ridge, param_grid=params)

In [93]:
grid.fit(X_train_scaled, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.1, 0.7, 1.0, 3.0, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [96]:
best = grid.best_estimator_
print(best)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)


In [98]:
best.fit(X_train_scaled, y_train)
pred = best.predict(X_test_scaled)
mse = mean_squared_error(pred, y_test)
np.sqrt(mse)

36472.836793861214

# Conclusion

Through identifying the best polynomial features via grid search, test/training and enhancing our model with multiple series, we have reduced the overall root mean square error by 32%.